In [3]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv('/content/BX-Books.csv',encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})
df1=pd.read_csv('/content/BX-Book-Ratings.csv',encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

FileNotFoundError: [Errno 2] No such file or directory: '/content/BX-Books.csv'

In [ ]:
df.head()

: 

In [ ]:
df1.head()

: 

In [ ]:
df.info()

: 

In [ ]:
df1.info()

: 

In [ ]:
df1.user.unique()

: 

In [ ]:
counts1 = df1['user'].value_counts()
counts2 = df1['isbn'].value_counts()

df1 = df1[~df1['user'].isin(counts1[counts1 < 200].index)]
df1 = df1[~df1['isbn'].isin(counts2[counts2 < 100].index)]

df1 = pd.merge(right=df1, left = df, on="isbn")

df1 = df1.drop_duplicates(["title", "user"])

piv = df1.pivot(index='title', columns='user', values='rating').fillna(0)

: 

In [ ]:
matrix = piv.values
matrix

: 

In [ ]:
matrix.shape

: 

In [ ]:
piv.head(50)

: 

In [ ]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(matrix)

: 

In [ ]:
# x=piv.loc['The Queen of the Damned (Vampire Chronicles (Paperback))'].array.reshape(1,-1)
# x

: 

In [ ]:
# distances,indices=model_knn.kneighbors(x,n_neighbors=6)
# indices

: 

In [ ]:
# R_books=[]
# for distances,indices in zip(distances[0],indices[0]):
#   if distances!=0:
#     R_book=piv.index[indices]
#     R_books.append([R_book,distances])
# recommended_books=['The Queen of the Damned (Vampire Chronicles (Paperback))',R_books]
# recommended_books

: 

In [ ]:
def get_recommends(book = ""):
  x=piv.loc[book].array.reshape(1, -1)
  distances,indices=model_knn.kneighbors(x,n_neighbors=6)
  R_books=[]
  for distance,indice in zip(distances[0],indices[0]):
    if distance!=0:
      R_book=piv.index[indice]
      R_books.append([R_book,distance])
  recommended_books=[book,R_books[::-1]]
  return recommended_books

: 

In [ ]:
get_recommends('Black Friday')

: 

In [ ]:
print(recommended_books)

: 

In [ ]:
books=get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

: 

In [ ]:
test_pass=True
recommends=get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
  test_pass=False
recommended_books=["I'll Be Seeing You","The Weight of Water","The Surgeon","I Know This Much Is True"]
recommended_books_dist=[0.8,0.77,0.77,0.77]
for i in range(2):
  if recommends[1][i][0] not in recommended_books:
    test_pass=False
  if abs(recommends[1][i][1]-recommended_books_dist[i])>=0.05:
    test_pass=False
if test_pass:
  print("passed")
else:
  print("failed")

: 

In [ ]:
import pickle
filename = 'book_recommend.pkl'
with open(filename, 'wb') as file:
    pickle.dump(recommended_books, file)

: 

: 